In [8]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dropout,Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical

In [9]:
img_width, img_height = 150, 150  

train_data_dir = '../dataset/split_data/train/'
validation_data_dir = '../dataset/split_data/validation/'
top_model_weights_path = 'bottleneck_fc_model.h5'
nb_train_samples = 13447
nb_validation_samples = 4487
nb_classes = 10
epochs = 3
batch_size = 16

In [10]:
def save_bottleneck_features():

    model = applications.VGG16(include_top=False, weights='imagenet')  
    datagen = ImageDataGenerator(rescale=1. / 255)  

    generator = datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), \
                                            batch_size=batch_size, class_mode=None, shuffle=False)  

    n_train_samples = len(generator.filenames)  
    n_classes = len(generator.class_indices)
    print("Number of train files = {}".format(n_train_samples))
    print("Number of classes = {}".format(n_classes))

    predict_size_train = int(math.ceil(n_train_samples / batch_size))  

    bottleneck_features_train = model.predict_generator(generator, predict_size_train, verbose=1)  

    np.save('bottleneck_features_train.npy', bottleneck_features_train) 

    generator = datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height), \
     batch_size=batch_size, class_mode=None, shuffle=False)  

    n_validation_samples = len(generator.filenames)  

    predict_size_validation = int(math.ceil(n_validation_samples / batch_size))  

    bottleneck_features_validation = model.predict_generator(generator, predict_size_validation, verbose=1)  

    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)  



In [11]:
def create_top_model(final_activation,input_shape):
    model = Sequential()  
    model.add(Flatten(input_shape=input_shape))  
    model.add(Dense(256, activation='relu'))  
    model.add(Dropout(0.5))  
    model.add(Dense(nb_classes, activation=final_activation)) # sigmoid to train, softmax for prediction
    return model

In [12]:
def train_top_model():
    datagen_top = ImageDataGenerator(rescale=1./255)  
    generator_top = datagen_top.flow_from_directory(train_data_dir, target_size=(img_width, img_height),\
                                                    batch_size=batch_size, class_mode='categorical', \
                                                    shuffle=False)  

    n_train_samples = len(generator_top.filenames)  
    n_classes = len(generator_top.class_indices)  

    # load the bottleneck features saved earlier  
    train_data = np.load('bottleneck_features_train.npy')  

    # get the class lebels for the training data, in the original order  
    train_labels = generator_top.classes  

    # convert the training labels to categorical vectors  
    train_labels = to_categorical(train_labels, num_classes=n_classes)

    generator_top = datagen_top.flow_from_directory(validation_data_dir, target_size=(img_width, img_height),\
                                                    batch_size=batch_size, class_mode=None, shuffle=False)  

    n_validation_samples = len(generator_top.filenames)  

    validation_data = np.load('bottleneck_features_validation.npy')

    validation_labels = generator_top.classes  
    validation_labels = to_categorical(validation_labels, num_classes=n_classes)

    model = create_top_model("sigmoid",train_data.shape[1:])

    model.compile(optimizer='rmsprop',  
              loss='categorical_crossentropy', metrics=['accuracy'])  

    history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size,\
                        validation_data=(validation_data, validation_labels))  

    model.save_weights(top_model_weights_path)  

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, \
                                                batch_size=batch_size, verbose=1)

    print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
    print("[INFO] Loss: {}".format(eval_loss))
    return model


In [13]:
def get_prediction_from_image(img_path):
    img_path = "../dataset/split_data/test/c5/img_18411.jpg"
    target_size=(150,150)

    # prepare image for classification using keras utility functions
    image = load_img(img_path, target_size=target_size)
    image = img_to_array(image) # convert from PIL Image to NumPy array
    image /= 255
    # the dimensions of image should now be (150, 150, 3)

    # to be able to pass it through the network and use batches, we want it with shape (1, 224, 224, 3)
    image = np.expand_dims(image, axis=0)
    print(image.shape)

    # build the VGG16 network  
    model = applications.VGG16(include_top=False, weights='imagenet')  

    # get the bottleneck prediction from the pre-trained VGG16 model  
    bottleneck_prediction = model.predict(image) 
    
    # build top model  
    model = create_top_model("softmax",bottleneck_prediction.shape[1:])

    model.load_weights(top_model_weights_path)  

    # use the bottleneck prediction on the top model to get the final classification  
    class_predicted = model.predict_classes(bottleneck_prediction) 
    probs = model.predict(bottleneck_prediction) 

    print(class_predicted)
    print(probs)

In [14]:
if __name__ == "__main__":
    #save_bottleneck_features() # Takes a lot of time
    model = train_top_model()
    get_prediction_from_image("../dataset/split_data/test/c5/img_18411.jpg")

Found 13447 images belonging to 10 classes.
Found 4487 images belonging to 10 classes.
Train on 13447 samples, validate on 4487 samples
Epoch 1/3
13447/13447 [==============================] - 12s 886us/step - loss: 1.5570 - acc: 0.4426 - val_loss: 0.5551 - val_acc: 0.7923
Epoch 2/3
13447/13447 [==============================] - 12s 873us/step - loss: 0.4260 - acc: 0.8647 - val_loss: 0.1029 - val_acc: 0.9724
Epoch 3/3
4487/4487 [==============================] - 1s 136us/step
[INFO] accuracy: 96.92%
[INFO] Loss: 0.10964528190558193
(1, 150, 150, 3)
[5]
[[  4.53916982e-05   4.89248997e-08   4.57145416e-06   3.07668117e-04
    1.43566649e-04   9.34469879e-01   7.11626047e-09   3.79093956e-09
    2.57027597e-04   6.47718757e-02]]
